In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

## Scraping function [](#2)

In [2]:
def scrape_article_ids(api_url, max_pages):
    job_id = []
    titles = []
    teasers = []
    companies = []
    locations = []
    categorys = []
    subCategories = []
    job_types = []
    salaries = []
    dateposted = []
    datedisplay = []
    for page_number in range(1, max_pages + 1):
        page_url = f'{api_url}&page={page_number}'

        # Send an HTTP request to the API endpoint
        response = requests.get(page_url)
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()

            # Extract advertiser IDs from each item in the 'data' list
            for item in data['data']:
                jid = item['id']
                title = item['title']
                teaser = item['teaser']
                company = item['advertiser'].get('description', '')
                location = item.get('location', '')
                category = item['classification'].get('description', '')
                subCategory= item['subClassification'].get('description', '')
                job_type = item.get('workType', '')
                salary = item.get('salary', '')
                date = item.get('listingDate', '')
                display = item.get('listingDateDisplay', '')

                job_id.append(jid)
                titles.append(title)
                teasers.append(teaser)
                companies.append(company)
                locations.append(location)
                categorys.append(category)
                subCategories.append(subCategory)
                job_types.append(job_type)
                salaries.append(salary)
                dateposted.append(date)
                datedisplay.append(display)

        else:
            print(f"Failed to retrieve data from the API. Status Code: {response.status_code}")
            break

    return job_id, titles, teasers, companies, locations, categorys, subCategories,job_types,salaries, dateposted, datedisplay

def combine_results(api_urls, max_pages):
    combined_data = {'job_id': [], 'titles': [], 'teasers': [], 'companies': [], 'locations': [], 'categorys': [],
                     'subCategories': [], 'job_types': [], 'salaries': [], 'dateposted': [], 'datedisplay': []}

    for api_url in api_urls:
        job_id, titles, teasers, companies, locations, categorys, subCategories, job_types, salaries, dateposted, datedisplay = scrape_article_ids(api_url, max_pages)
        combined_data['job_id'].extend(job_id)
        combined_data['titles'].extend(titles)
        combined_data['teasers'].extend(teasers)
        combined_data['companies'].extend(companies)
        combined_data['locations'].extend(locations)
        combined_data['categorys'].extend(categorys)
        combined_data['subCategories'].extend(subCategories)
        combined_data['job_types'].extend(job_types)
        combined_data['salaries'].extend(salaries)
        combined_data['dateposted'].extend(dateposted)
        combined_data['datedisplay'].extend(datedisplay)

    return combined_data

max_pages = 7

# Not sure how long the api_urls can last for
api_urls = [
    'https://www.jobstreet.com.sg/api/chalice-search/v4/search?siteKey=SG-Main&sourcesystem=houston&userqueryid=a807f6a20a0666f3b89aa9e3cb3bca23-4784291&userid=627c24a7-8b33-456a-9420-ed94dd9c1da6&usersessionid=627c24a7-8b33-456a-9420-ed94dd9c1da6&eventCaptureSessionId=627c24a7-8b33-456a-9420-ed94dd9c1da6&where=singapore&seekSelectAllPages=true&keywords=data+analytics&pageSize=30&include=seodata&locale=en-SG&seekerId=527177449&solId=82238ce8-3314-4947-a2bb-606e4dce4c48',
    'https://www.jobstreet.com.sg/api/chalice-search/v4/search?siteKey=SG-Main&sourcesystem=houston&userqueryid=9fe593460e74b18f4ff43b70ea18f3b2-7061267&userid=627c24a7-8b33-456a-9420-ed94dd9c1da6&usersessionid=627c24a7-8b33-456a-9420-ed94dd9c1da6&eventCaptureSessionId=627c24a7-8b33-456a-9420-ed94dd9c1da6&where=singapore&seekSelectAllPages=true&keywords=data+science&pageSize=30&include=seodata&locale=en-SG&seekerId=527177449&solId=82238ce8-3314-4947-a2bb-606e4dce4c48',
    'https://www.jobstreet.com.sg/api/chalice-search/v4/search?siteKey=SG-Main&sourcesystem=houston&userqueryid=7ad56fc8a31dcbb7878db4b353af1c3e-7122654&userid=627c24a7-8b33-456a-9420-ed94dd9c1da6&usersessionid=627c24a7-8b33-456a-9420-ed94dd9c1da6&eventCaptureSessionId=627c24a7-8b33-456a-9420-ed94dd9c1da6&where=singapore&seekSelectAllPages=true&keywords=AI&pageSize=30&include=seodata&locale=en-SG&seekerId=527177449&solId=82238ce8-3314-4947-a2bb-606e4dce4c48',
    'https://www.jobstreet.com.sg/api/chalice-search/v4/search?siteKey=SG-Main&sourcesystem=houston&userqueryid=aca0e1278d6f7a2ee718037b5200293c-7195166&userid=627c24a7-8b33-456a-9420-ed94dd9c1da6&usersessionid=627c24a7-8b33-456a-9420-ed94dd9c1da6&eventCaptureSessionId=627c24a7-8b33-456a-9420-ed94dd9c1da6&where=singapore&seekSelectAllPages=true&keywords=machine+learning&pageSize=30&include=seodata&locale=en-SG&seekerId=527177449&solId=82238ce8-3314-4947-a2bb-606e4dce4c48'
]


## Fetch job descriptions from individual job link

In [3]:
def fetch_job_article(job_id):
    article_url = f'https://www.jobstreet.com.sg/job/{job_id}'
    response = requests.get(article_url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve job article. Status Code: {response.status_code}")
        return None

def extract_text_from_ul(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    ul_tags = soup.find_all('ul')
    text_list = [ul.get_text(separator='\n') for ul in ul_tags]
    return '\n'.join(text_list)

def scrape_and_store_text(job_ids):
    data = {'job_id': [],'job_title':[], 'job_teaser':[], 'company':[], 'descriptions': [], 'location':[],'category':[],'subcategory':[],'type':[],'salary':[],'dateposted':[],'datedisplay':[]}

    for job_id in job_ids:
        job_article_content = fetch_job_article(job_id)

        if job_article_content:
            text_from_ul = extract_text_from_ul(job_article_content)
            data['job_id'].append(job_id)
            data['descriptions'].append(text_from_ul)

    data['job_title'] = titles
    data['job_teaser'] = teasers
    data['company'] = companies
    data['location'] = locations
    data['category'] = categorys
    data['subcategory'] = subCategories
    data['type']=job_types
    data['salary']=salaries
    data['dateposted']=dateposted
    data['datedisplay']=datedisplay


    return data


## Combine API queries and output to CSV file

In [4]:
combined_data = combine_results(api_urls, max_pages)

result_df = pd.DataFrame(combined_data)

result_df.drop_duplicates(inplace=True)

result_df.to_csv('jobstreet_jobs_data.csv', index=False)

result_df

,job_id,titles,teasers,companies,locations,categorys,subCategories,job_types,salaries,dateposted,datedisplay
0,75321199,"Data Analyst - Marketing Analytics, Regional B...",-,SHOPEE SINGAPORE PRIVATE LIMITED,Central Region,Marketing & Communications,Marketing Communications,Full time,,2024-04-22T07:53:08Z,6h ago
1,75302666,Information Technology - Senior Data Engineer ...,We have multiple senior data engineering posit...,Singapore Airlines,East Region,Science & Technology,"Mathematics, Statistics & Information Sciences",Full time,,2024-04-21T14:39:00Z,23h ago
2,75319768,Data Analyst,You will be given good learning exposure.,Methodist Welfare Services,West Region,Healthcare & Medical,Clinical/Medical Research,Full time,,2024-04-22T07:05:05Z,7h ago
3,75246968,"Data Analytics Intern - Business Intelligence,...",-,SHOPEE SINGAPORE PRIVATE LIMITED,Central Region,Information & Communication Technology,Business/Systems Analysts,Full time,,2024-04-19T01:42:39Z,3d ago
4,75230897,Data Analyst Intern - Regional Marketing Analy...,-,SHOPEE SINGAPORE PRIVATE LIMITED,Central Region,Marketing & Communications,Market Research & Analysis,Full time,,2024-04-18T08:38:37Z,4d ago
...,...,...,...,...,...,...,...,...,...,...,...
886,75145123,FE GLOBAL PLANNING SIC,"• Thorough knowledge of Micron’s Lot Tracking,...",MICRON SEMICONDUCTOR ASIA OPERATIONS PTE LTD,Central Region,Engineering,Electrical/Electronic Engineering,Full time,,2024-04-15T19:17:00Z,6d ago
888,75190698,Site Reliability Engineer Manager (Big Data) -...,-,SHOPEE SINGAPORE PRIVATE LIMITED,Central Region,Information & Communication Technology,Management,Full time,,2024-04-17T05:24:56Z,5d ago
890,75151459,Robotics Engineer,"As an advocate for diversity & inclusion, Omni...",Omni-Health Pte Ltd.,Central Region,Engineering,Electrical/Electronic Engineering,Full time,,2024-04-16T02:35:04Z,6d ago
891,75270683,Technical Product Manager - Marketplace Intell...,-,SHOPEE SINGAPORE PRIVATE LIMITED,Singapore,Information & Communication Technology,Product Management & Development,Full time,,2024-04-19T07:02:24Z,3d ago
